# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

First, we import everything we will need later.

In [ ]:
#imports
import pandas as pd
import datetime
from dateutil.parser import parse
from os import listdir
from os.path import isfile, join

Then, we define some helpful functions that will help us during the parsing of the data.
- get_files: returns all the .csv files for a given country

In [ ]:
def get_files(country):
    path = DATA_FOLDER + "/ebola/" + country + "_data/"
    return [f for f in listdir(path) if isfile(join(path, f))]

- sum_row: for a given row, returns the total value for the new cases / deaths. We first defined this function as the sum of all new cases / deaths in all provinces, but we discovered some strange data for some provinces, so we decided to only take into account the 'total' column
- sum_rows: sum all the rows given in argument

In [ ]:
def sum_row(row, total_col):
    return float(row[total_col].values[0])

def sum_rows(rows, total_col):
    tot = 0
    for row in rows:
        tot += sum_row(row, total_col)
    return tot 

Now, we define for each country a function, which, for a given file, returns a dictionnary with the country, date, upper and lower bounds for the new cases, and upper and lower bounds for the new deaths.
As we don't know if the new cases / deaths for the 'probable' and 'suspected' cases is reliable, we decided to create an upper bound with the sum of the 'confirmed', 'probable' and 'suspected' new cases / deaths, and a lower bound with only the 'confirmed' new cases / deaths.

The structure of these functions are the same for each country, only the name of the descrption of the data changes.

In [ ]:
def get_row_guinea(file):
    country = 'guinea'
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file)
    total_col = "Totals"
    
    new_cases_lower = sum_row(raw[raw.Description == "New cases of confirmed"], total_col)
    new_cases_upper = sum_row(raw[raw.Description == "Total new cases registered so far"], total_col)
        
    new_deaths_lower = sum_row(raw[(raw.Description == "New deaths registered today (confirmed)") | (raw.Description == "New deaths registered")], total_col)
    new_deaths_upper = sum_row(raw[(raw.Description == "New deaths registered today") | (raw.Description == "New deaths registered")], total_col)
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'NewDeathsLower' : new_deaths_lower, 'NewDeathsUpper' : new_deaths_upper}

In [ ]:

def get_row_liberia(file):
    country = 'liberia'
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file).fillna(0)
    total_col = "National"
    
    new_cases_lower = sum_row(raw[raw.Variable == "New case/s (confirmed)"], total_col)
    list_cases_upper = (["New Case/s (Suspected)", 
                        "New Case/s (Probable)",
                        "New case/s (confirmed)"])
    new_cases_upper = sum_rows([raw[raw.Variable == row] for row in list_cases_upper], total_col)
        
    new_deaths_lower = sum_row(raw[raw.Variable == "Newly reported deaths"], total_col)
    new_deaths_upper = new_deaths_lower
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'NewDeathsLower' : new_deaths_lower, 'NewDeathsUpper' : new_deaths_upper}

As the files for the Sierra Leone does not contain data for the new deaths, we first extract the total deaths for each day, and we will process them later to get the new deaths.

In [ ]:
def get_row_sl(file):
    country = 'sl'
    date = file[:10]
    raw = pd.read_csv(DATA_FOLDER + "/ebola/" + country + "_data/" + file).fillna(0)
    total_col = "National"
        
    new_cases_lower = sum_row(raw[raw.variable == "new_confirmed"], total_col)
    list_cases_upper = (["new_suspected", 
                        "new_probable",
                        "new_confirmed"])
    new_cases_upper = sum_rows([raw[raw.variable == row] for row in list_cases_upper], total_col)
    
    list_death_upper = (["death_suspected", 
                        "death_probable",
                        "death_confirmed"])
    total_death_upper = sum_rows([raw[raw.variable == row] for row in list_death_upper], total_col)
    total_death_lower = sum_row(raw[raw.variable == "death_confirmed"], total_col)
    
    return {'Country' : country, 'Date' : parse(date), 'NewCasesLower' : new_cases_lower, 'NewCasesUpper' : new_cases_upper, 'TotalDeathLower' : total_death_lower, 'TotalDeathUpper' : total_death_upper}

In [ ]:
rows_guinea = [get_row_guinea(file) for file in get_files("guinea")]

In [ ]:
rows_liberia = [get_row_liberia(file) for file in get_files("liberia")]

We now transform the data for the Sierra Leone :
-we first create a new dictionary for which the keys are date, and the values are the previously extracted values from the .csv files.
-then for each value in this dictionary, we try to get the value of the day before, and perform the difference to get the new deaths of this day

In [ ]:
rows_sl_total_deaths = [get_row_sl(file) for file in get_files("sl")]
dic_sl_total_deaths = {}
for row in rows_sl_total_deaths:
    dic_sl_total_deaths[row['Date']] = row
    
rows_sl = []
for date, entry in dic_sl_total_deaths.items():
    date_before = date - datetime.timedelta(days=1)
    if date_before in dic_sl_total_deaths:
        
        if entry['TotalDeathUpper'] != 0 and dic_sl_total_deaths[date_before]['TotalDeathUpper'] != 0 and entry['TotalDeathLower'] != 0 and dic_sl_total_deaths[date_before]['TotalDeathLower'] != 0: 
            copy = dict(entry)
            del copy['TotalDeathUpper']
            del copy['TotalDeathLower']
            
            copy['NewDeathsUpper'] = entry['TotalDeathUpper'] - dic_sl_total_deaths[date_before]['TotalDeathUpper']
            copy['NewDeathsLower'] = entry['TotalDeathLower'] - dic_sl_total_deaths[date_before]['TotalDeathLower']

            rows_sl.append(copy)

We can now insert the data in a dataframe

In [ ]:
raw_dataframe = pd.DataFrame(columns=['Country', 'Date', 'NewCasesLower', 'NewCasesUpper', 'NewDeathsLower', 'NewDeathsUpper'])
for row in rows_sl, rows_guinea:
    raw_dataframe = raw_dataframe.append(row, ignore_index = True)
for row in rows_liberia:
    if row['Date'].month != 12: #December data is erroneous
        raw_dataframe = raw_dataframe.append(row, ignore_index = True)
        
raw_dataframe

In [ ]:
dataframe = raw_dataframe.set_index(['Country', 'Date'])

In [ ]:
dataframe_no_day = raw_dataframe
dataframe_no_day['Year'] = raw_dataframe['Date'].apply(lambda x: x.year)
dataframe_no_day['Month'] = raw_dataframe['Date'].apply(lambda x: x.month)
final_df = dataframe_no_day[['Country', 'Year', 'Month', 'NewCasesLower', 'NewCasesUpper', 'NewDeathsLower', 'NewDeathsUpper']].groupby(['Country', 'Year', 'Month']).mean()
final_df

Finally, to have some final general idea for the data, we average the bounds.

In [ ]:
s1 = final_df[['NewCasesLower', 'NewCasesUpper']].mean(axis=1)
s2 = final_df[['NewDeathsLower', 'NewDeathsUpper']].mean(axis=1)
final = pd.concat([s1, s2], axis=1)
final.columns = ['NewCasesAverage', 'NewDeathsAverage']
final

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

We start by importing pandas.

In [ ]:
import pandas as pd

We load the first spreadsheet from the file's Sheet 1. Then we add a new column that is the same for all the data in this import, which corresponds to the barcode of the code.

Then we rename the columns for more clarity.

In [ ]:
mid = pd.read_excel(DATA_FOLDER + '/microbiome/MID1.xls', sheetname='Sheet 1', header=None)
mid.fillna('unknown', inplace=True)
mid['BARCODE'] = 'MID1'
mid.columns = ['Taxon', 'Count', 'BARCODE']

Now we repeat this operation for every other spreadsheet except the metadata. At each iteration we simply concatenate the data at the end of the previous data, this accumulating all the files' data into a single dataframe. We don't care about any index right now since we will use a random one later.

In [ ]:
for i in range(2, 10):
    midi = pd.read_excel(DATA_FOLDER + '/microbiome/MID' + str(i) + '.xls', sheetname='Sheet 1', header=None)
    midi.fillna('unknown', inplace=True)
    midi['BARCODE'] = 'MID' + str(i)
    midi.columns = ['Taxon', 'Count', 'BARCODE']
    mid = pd.concat([mid, midi])

Finally, we do a merge with the metadata. We join on the `BARCODE` column. This column will be the index of the metadata when we import it in this case. Finally we set the index for the three columns `BARCODE`, `GROUP` and `SAMPLE` which are all the columns of the metada and are unique.

The only `NaN` value we found was the NA value on the metadata, which may indicate that there is no sample for the first group. We replaced it anyway by `unknown`.

In [ ]:
metadata = pd.read_excel(DATA_FOLDER + '/microbiome/metadata.xls', sheetname='Sheet1', index_col=0)
metadata.fillna('unknown', inplace=True)
merged = pd.merge(mid, metadata, right_index=True, left_on='BARCODE')
merged = merged.set_index(keys=['BARCODE', 'Taxon'])
merged

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here